# Accessibility to schools in rural areas

In [ ]:
# If using colab
# Takes around 2-3 min
!pip install git+https://github.com/CityScope/UrbanAccessAnalyzer.git
!pip install matplotlib mapclassify folium
!apt-get install -y osmium-tool

# Restart notebook after installing this if needed

  Cloning https://github.com/CityScope/UrbanAccessAnalyzer.git to /tmp/pip-req-build-3z21csfo
  Running command git clone --filter=blob:none --quiet https://github.com/CityScope/UrbanAccessAnalyzer.git /tmp/pip-req-build-3z21csfo
  Resolved https://github.com/CityScope/UrbanAccessAnalyzer.git to commit a79f5923261faa3e9fe066ec03b7da5e302ef1f1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 55.4 MB/s eta 0:00:00
  Created wheel for UrbanAccessAnalyzer: filename=urbanaccessanalyzer-0.1.0-py3-none-any.whl size=45680 sha256=821c5f2f55f550d3a577408db682b017789bce15216d6e43101c37396bc924ba
  Stored in directory: /tmp/pip

In [ ]:
import os
from datetime import datetime, date, timedelta, time
import pandas as pd
import geopandas as gpd
import os

import osmnx as ox

import matplotlib.pyplot as plt

import UrbanAccessAnalyzer.isochrones as isochrones
import UrbanAccessAnalyzer.graph_processing as graph_processing
import UrbanAccessAnalyzer.osm as osm
import UrbanAccessAnalyzer.utils as utils
import UrbanAccessAnalyzer.population as population

## 1 Points of interest

Points of interest (poi): Schools as point geometry

Area of interest (aoi): Polygon to do the analisys

### 1.1 Results folder

In [ ]:
results_path = os.path.normpath("results")
os.makedirs(results_path,exist_ok=True)

### 1.2 Area of interest

Option 1: Write the city name

In [ ]:
city_name = "Valladolid, España"
city_filename = utils.sanitize_filename(city_name)
aoi = utils.get_city_geometry(city_name)
geo_suggestions = utils.get_geographic_suggestions_from_string(city_name,user_agent="app")
geo_suggestions

{'country_codes': ['ES'],
 'subdivision_names': ['Castile and León', 'Valladolid'],
 'municipalities': ['Valladolid']}

Option 2: Load your own file (.gpkg or .shp)

In [ ]:
# aoi = gpd.read_file("")
# city_name = ""

Use UTM coords and creat aoi_download with a buffer of X meters

In [ ]:
aoi = gpd.GeoDataFrame(geometry=[aoi.union_all()],crs=aoi.crs) # Ensure there is only one polygon
aoi = aoi.to_crs(aoi.estimate_utm_crs()) # Convert to utm
aoi_download = aoi.buffer(20000) # Area to do streets and poi requests

Map of your area of interest and the download area (aoi_buffer)

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m

### 1.3 OpenStreetMap

In [ ]:
query = """
[out:xml] [timeout:25];
(
    node["amenity"="school"]( {{bbox}});
    way["amenity"="school"]( {{bbox}});
    relation["amenity"="school"]( {{bbox}});
);
(._;>;);
out body;
"""

In [ ]:
poi = osm.overpass_api_query(query,aoi_download)
poi.geometry = poi.geometry.centroid
poi = poi.to_crs(aoi.crs)
poi

,geometry,type,id,nodes,addr:city,addr:postcode,addr:street,amenity,name,nohousenumber,...,contact:twitter,addr:place,pedagogy,fence,name:en,opening_hours,old_name,internet_access,wheelchair,geometry_type
0,POINT (359588.212 4617020.359),node,3100622795,NaN,Santovenia de Pisuerga,47155,Calle Miguel de Cervantes,school,Colegio Público Nicómedes Sanz,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (355564.488 4611279.992),node,3270028748,NaN,Valladolid,47007,Calle de Gabilondo,school,Centro de Enseñanza Concertado Gregorio Fernández,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (357889.812 4607898.139),node,3718312321,NaN,Valladolid,47012,Calle del Plomo,school,LYCEUM Formación,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (372167.672 4599284.798),node,3842785951,NaN,NaN,NaN,NaN,school,C.R.A. La Parrilla,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (355971.055 4610443.579),node,4503252691,NaN,NaN,NaN,NaN,school,La Escuela de Diseño - ESI Valladolid,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,POINT (355931.68 4608663.36),relation,4078971,NaN,Valladolid,47008,Avenida de Madrid,school,Colegio San Agustín,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon
213,POINT (350191.076 4606839.216),relation,4285327,NaN,Simancas,47130,Camino Viejo de Simancas,school,Colegio Pinoalbar,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon
214,POINT (355220.157 4613430.938),relation,4297589,NaN,Valladolid,47009,Calle de la Sementera,school,Instituto de Educación Secundaria Emilio Ferrari,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,multipolygon
215,POINT (355074.045 4609826.281),relation,4620208,NaN,Valladolid,47008,Calle del Doctor Moreno,school,Colegio de Educación Infantil Vicente Aleixandre,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,multipolygon


Map with points of interest (schools), download area and area of interest

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m=poi.explore(m=m,color='blue')
m

## 2 Street graph

### 2.1 Regionwise file and cropping

- Download best regionwise pbf file. (Covers a large area)

- Crop it to cover our area of interest and save it in .osm format

In [ ]:
osm_xml_file = os.path.normpath(results_path+"/streets.osm")
streets_graph_path = os.path.normpath(results_path+"/streets.graphml")
streets_path = os.path.normpath(results_path+"/streets.gpkg")
level_of_service_streets_path = os.path.normpath(results_path+"/level_of_service_streets.gpkg")

In [ ]:
# Select what type of street network you want to load
network_filter = osm.osmium_network_filter("walk+bike+primary")
# Download the region pbf file crop it by aoi and convert to osm format
osm.geofabrik_to_osm(
    osm_xml_file,
    input_file=results_path,
    aoi=aoi_download,
    osmium_filter_args=network_filter,
    overwrite=False
)

File results does not exist. Downloading best matching geofabrik file.
Fetching Geofabrik index from https://download.geofabrik.de/index-v1.json...
Downloaded geofabrik to /content/results/castilla_y_leon.osm.pbf
Applying tag filter: w/highway=trunk_link w/highway=steps w/highway=trunk w/highway=secondary w/highway=residential w/highway=footway w/highway=primary_link w/highway=pedestrian w/highway=service w/highway=path w/highway=living_street w/highway=unclassified w/highway=track w/highway=tertiary_link w/highway=cycleway w/highway=primary w/highway=tertiary w/highway=secondary_link w/foot=yes w/bicycle=yes
Creating .poly file for AOI clipping...
Extracting by geometry...
Finished. Final output: results/streets.osm


'results/streets.osm'

### 2.2 Load to osmnx

This way the street network is a networkx graph

In [ ]:
# Load
G = ox.graph_from_xml(osm_xml_file)
# Project geometry coordinates to UTM system to allow euclidean meassurements in meters (sorry americans)
G = ox.project_graph(G,to_crs=aoi.estimate_utm_crs())
# Save the graph in graphml format to avoid the slow loading process
ox.save_graphml(G,streets_graph_path)

### 2.3 Simplify graph

Edges with length smaler than X meters are deleted and its nodes merged

In [ ]:
min_edge_length = 30

G = graph_processing.simplify_graph(G,min_edge_length=min_edge_length,min_edge_separation=min_edge_length*2,undirected=True)
# Save the result in graphml format
ox.save_graphml(G,streets_graph_path)

### 2.4 Points of interest

Add pois to street graph

In [ ]:
G, osmids = graph_processing.add_points_to_graph(
    poi,
    G,
    max_dist=100+min_edge_length, # Maximum distance from point to graph edge to project the point
    min_edge_length=min_edge_length # Minimum edge length after adding the new nodes
)
poi['osmid'] = osmids # Add the ids of the nodes in the graph to points

## 3 Compute isochrones

### 3.1 Distance steps

In [ ]:
distance_steps = [1000,3000,20000]
level_of_services = ['walk','bike','bus']

In [ ]:
level_of_service_graph = isochrones.graph(
    G,
    poi,
    distance_steps, # If service_quality_col is None it could be a list of distances
    service_quality_col = None, # If all points have the same quality this could be None
    level_of_services = level_of_services, # could be None and it will set to the sorted unique values of the matrix
    min_edge_length = min_edge_length # Do not add new nodes if there will be an edge with less than this length
)
# Save edges as gpkg
level_of_service_nodes, level_of_service_edges = ox.graph_to_gdfs(level_of_service_graph)
level_of_service_edges.to_file(level_of_service_streets_path)

  0%|          | 0/3 [00:00<?, ?it/s]


TypeError: only list-like objects are allowed to be passed to isin(), you passed a `int`

#### Lets visualize the results on a map

In [ ]:
m = level_of_service_edges.explore(
    column='level_of_service',
    cmap="RdYlGn_r",
)

m = poi[[
    "name",
    "geometry"
]].explore(
    m=m,
    color="black",
    style_kwds={
        "color": "black",       # Border color
        "weight": 1,            # Border thickness
        "opacity": 1.0,         # Border opacity
        "fillOpacity": 1,
        "radius": 6,
    },
)

m.save(results_path + "/level_of_service_streets.html")
# If map does not render
# import webbrowser
# webbrowser.open(results_path + "/PToffer_map.html")
m

## 4 Population

### 4.1 Download Worldpop tif file

- One file for every country
- 100m pixel size
- tif format
- available from 2000 to 2030
- gender and age

In [ ]:
population_file = population.download_worldpop_population(
    aoi_download,
    2025,
    folder=results_path,
    resolution="100m",
    dataset="age_structures",
    subset="U18"
)

### 4.2 Filter population grid by streets

The population files are not very precise in location. Sometimes population appears in inaccessible places. This makes the results slightly better.

In [ ]:
street_edges = gpd.read_file(level_of_service_streets_path)
pop_raster, pop_transform, pop_crs = population.filter_population_by_streets(
    streets_gdf=street_edges,
    population=population_file,
    street_buffer=25,
    aoi=aoi_download,
    min_population=1
)

### 4.3 Assign level of service to each population cell

In [ ]:
pop_raster, transform, crs = raster_utils.read_raster(population,aoi=aoi,nodata=0)
level_of_service_edges = level_of_service_edges.to_crs(level_of_service_edges.estimate_utm_crs())
level_of_service_edges.geometry = level_of_service_edges.geometry.simplify(street_buffer/2).buffer(street_buffer,resolution=4)
level_of_service_raster = raster_utils.rasterize(
    gdf=level_of_service_edges,
    shape=pop_raster,
    transform=transform,
    crs=crs,
    value_column=level_of_service_column,
    value_order=level_of_services
)

pop_gdf = raster_utils.vectorize(pop_raster,transform,crs,keep_nodata=True,nodata=0,min_value=1)
pop_gdf = pop_gdf.rename(columns={'value':'population'})
pop_gdf['population'] = pop_gdf['population'].astype(float).fillna(0)
pop_gdf['level_of_service'] = level_of_service_raster.flatten()
pop_gdf = pop_gdf[['id','population', *pop_gdf.columns[3:],'geometry']]
pop_gdf = pop_gdf[pop_gdf['population'] > 1].reset_index(drop=True)
pop_gdf.to_file(os.path.normpath(results_path + "/population.gpkg"))
